In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pmdarima
from pmdarima import auto_arima

In [ ]:
train = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/train.csv')
test = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/test.csv')
samp = pd.read_csv('/kaggle/input/demand-forecasting-kernels-only/sample_submission.csv')

In [ ]:
import multiprocessing as mp
from joblib import Parallel, delayed

In [ ]:
def applyParallel(dfGrouped, func):
    '''
    Purpose: Parallel processing module
    Inputs: Grouped data at forecast level
    Output: DataFrame with corresponding feature added
    '''
    print("Applying parallel ...")
    retLst = Parallel(n_jobs=mp.cpu_count())(delayed(func)(group) for name, group in dfGrouped)
    print(retLst)
    return# pd.concat(retLst)

In [ ]:
mp.cpu_count()

In [ ]:
train.head()

In [ ]:
# the_store = 1
# the_item = 2

In [ ]:
!pip install -i https://test.pypi.org/simple/ common-ds-utils==1.0.2

In [ ]:
from common_ds_utils.metrics.mean_absolute_percentage_error import mean_absolute_percentage_error

In [ ]:
def run_arima(df):
    
    print("store : {}".format(df.store.iloc[0]))
    print("item : {}".format(df.item.iloc[0]))
    
    train = df[df['date'] < '2017-01-01']
    
    valid = df[df['date'] >= '2017-01-01']
    
    arima_model = auto_arima(train['sales'])
    
    valid['pred'] = arima_model.predict(n_periods=len(valid))
    
    mape = mean_absolute_percentage_error(valid['sales'], valid['pred'])
    
    print("valid mape : {}".format(mape))
    
    result = {(df.store.iloc[0], df.item.iloc[0]):mape}
    
    return result

In [ ]:
sel_stores = train.store.unique()[:1]
sel_items = train.item.unique()[:4]

In [ ]:
temp = train[(train.store.isin(sel_stores)) & (train.item.isin(sel_items))]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
# Conventional for loop
for name, group in temp.groupby(['store', 'item']):
    run_arima(group)
    print("Model trained for {}".format(name))
    print("-------")
print("Hooray ...")

In [ ]:
%%time
applyParallel(temp.groupby(['store', 'item']), run_arima)